In [1]:
!pip install sentence_transformers --quiet

import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
import sentence_transformers
from sentence_transformers import SentenceTransformer, util, CrossEncoder
import os
import random
import time
import torch
import torch.nn as nn
import numpy as np
from torch.nn.functional import binary_cross_entropy_with_logits as bce_loss

import time
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.2 MB/s eta 0:00:00


In [2]:
!pip3 install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.4 MB/s eta 0:00:00


In [3]:
from pdfminer.high_level import extract_text
 
text = extract_text("/content/paper1.pdf")
output_file = open("/content/Output.txt", "w")
output_file.write(text)
output_file.close()

In [4]:
import csv, os, sys
import glob

with open('sample_input_paragraph.csv', 'w', newline="", encoding="utf-16") as out_file:
    csv_out = csv.writer(out_file)
    count=1
    csv_out.writerow(['id', 'paragraph','theme','title'])
    for text_file in glob.iglob('Output.txt'):
        with open(text_file, 'r') as txt:
            for line in txt.read().split('\n\n'):
                csv_out.writerow([count, line,'Material Science',"The Mechanical Properties of Natural Materials. I. Material Property Charts"])
                count+=1

In [5]:
###### PASSAGE ENCODING #########
bi_encoder = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")

para = pd.read_csv("/content/sample_input_paragraph.csv", encoding="utf-16")

for index,row in para.iterrows():
  if len(row["paragraph"])<20:
    para=para.drop(index)

grouped_para = dict(para.groupby('theme')['id'].apply(list))

curr_pairs = grouped_para.items()

for k,v in curr_pairs:
  passages = list(para[para['id'].isin(v)]['paragraph'])
  embeddings = bi_encoder.encode(passages)
  grouped_para[k] = [v, embeddings]

new_dict = {}

for k,v in grouped_para.items():
  new_dict[k.lower()] = v

grouped_para = new_dict

##### RE-RANKER MODEL ##########
cross_encoder = CrossEncoder("anaghasavit/ms-marco-TinyBERT-L-2-v2-20epochs")

###### QA AND DECIDER MODEL  ###################
model_checkpoint="anaghasavit/qa-finetuned"
question_answerer = pipeline("question-answering", model=model_checkpoint)

###### FINAL NECESSITY ############ 
paragraph_ds = grouped_para

global_model = {"bi_encoder":bi_encoder,
                "cross_encoder":cross_encoder,
                # "decider":decider,
                "qa":question_answerer}


In [12]:
def get_model(theme):
  return global_model

In [13]:
def pred_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"].lower()
  for question in questions:
    docs = np.array(paragraph_ds[theme][0])
    doc_all_embs = paragraph_ds[theme][1]
    
    top_n = min(15, len(doc_all_embs))
    top_cross = min(3, top_n)
    
    query_embedding = theme_model["bi_encoder"].encode(question["question"])
    scores = util.dot_score(query_embedding, doc_all_embs).cpu()
    top_k_indices = torch.topk(scores, k=top_n, dim=-1).indices

    top_k_passage = para[para["id"].isin(docs[top_k_indices[0]])]['paragraph'].values

    model_inputs = [[question["question"], p] for p in top_k_passage]

    scores = theme_model["cross_encoder"].predict(model_inputs)
    top_indices = torch.topk(torch.tensor(scores),top_cross)[1]
    
    best_n_passage = [model_inputs[top_indices[i]][1] for i in range(top_cross)]
    
    prev_best = 0
    curr_pass = -1
    curr_ans = ""
    for best_one in best_n_passage:
      res = question_answerer(question=question["question"], context=best_one)
      # print(question)
      # print(res)
      if res['score'] > 0.2 and res['score'] > prev_best:
        prev_best = res['score']
        curr_ans = res['answer']
        curr_pass = int(para[para["paragraph"]==best_one]['id'])
   
        

    ans = {}
    ans["question_id"] = question["id"]
    
    ans["paragraph_id"] = curr_pass
    ans["answers"] = curr_ans
    pred_out.append(ans)

In [14]:
questions=[
           "What are the strengths of silks?",
          #  "What does wood consist of?",
          #  "What is the strength of Kevlar?",
          #  "What is the stiffness of Kevlar?",
           ]

In [15]:
with open('sample_theme_interval.csv', 'w', newline="", encoding="utf-16") as out_file:
    csv_out = csv.writer(out_file)
    count=1
    csv_out.writerow(['theme', 'start','end'])
    for question in questions:
      csv_out.writerow(['Material Science', 1,205])
      count+=1

In [16]:
import time


with open('sample_input_question.csv', 'w', newline="", encoding="utf-16") as out_file:
    csv_out = csv.writer(out_file)
    count=1
    csv_out.writerow(['id', 'question','theme'])
    for question in questions:
      csv_out.writerow([count, question,'Material Science'])
      count+=1
st=time.time()
# All theme prediction.
questions = json.loads(pd.read_csv("sample_input_question.csv",encoding="utf-16").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("sample_theme_interval.csv",encoding="utf-16").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  theme_model = get_model(theme)
  execution_time = timeit.timeit(lambda: pred_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.to_csv('sample_output_prediction.csv', index=False)
et=time.time()
if pred_df["paragraph_id"][0]==-1:
  print("Answer not possible")
else:
  id=pred_df["paragraph_id"][0]
  for index,row in para.iterrows():
    if row["id"]==id:
      paragraph=row["paragraph"]
      paper=row["title"]
  print("answer:",pred_df["answers"][0])
  print("paper:",paper)
  print("paragraph:", paragraph)

answer: 0.3-2.0 GPa
paper: The Mechanical Properties of Natural Materials. I. Material Property Charts
paragraph:  Natural polymers and polymer composites include cellulose, chitin, silk, cuti-
 cle, collagen, keratin and tendon. All have densities of around 1.2 Mg m-3. Their
 moduli and tensile strengths are larger than those of engineering polymers: cel-
 lulose fibrils, for instance, have moduli of about 50-130 GPa and a strength of
 1 GPa, and silks have moduli of 2-20 GPa and strengths of 0.3-2.0 GPa. Of man-
 made polymers only Kevlar fibre has a higher stiffness (200 GPa) and strength
 (up to 4 GPa) which it achieves, as do natural fibres, through its highly oriented
 molecular structure.
